# Preprocess acoustic data and sync to neural data, one session at a time

This notebook is a modified version of the *1-preprocess_acoustics* in the chronic ephys processing pipeline

If *1-preprocess_acoustics* exits with errors, this notebook allows you to make manual adjustments

Common errors include:
- data streams cannot be synched (ex. neural and audio data streams are of different lengths)
- TTL events were skipped (i.e., the machine clock malfunctioned or SpikeGLX crashed and the data streams terminated at different moments)

Use the environment **songproc** to run this notebook

(currently using environment spikesort)

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import os
import pickle
from scipy.io import wavfile
import traceback

import sys
sys.path.append('/mnt/cube/lo/envs/ceciestunepipe')
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb
from ceciestunepipe.util import stimutil as su
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import sglxsync as sy
from ceciestunepipe.mods import sglxsync_debug as syd
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex
from ceciestunepipe.util import oeutil as oeu
from ceciestunepipe.mods import preproc_sglx, preproc_oe

import logging
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.WARNING) # set to logging.INFO if you'd like to see the full readout

/mnt/cube/lo/envs/songproc/lib/python3.8/site-packages/spikeextractors/__init__.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if StrictVersion(h5py.__version__) > '2.10.0':
2024-08-14 12:19:41,818 root         INFO     Running on pakhi.ucsd.edu


h5py version > 2.10.0. Some extractors might not work properly. It is recommended to downgrade to version 2.10.0: 
>>> pip install h5py==2.10.0


## Set parameters

In [2]:
## default bout detection parameters that work well for zebra finches
hparams = {
    # spectrogram
    'num_freq':1024, # how many channels to use in a spectrogram
    'preemphasis':0.97,
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spectrogram
    'ref_level_db':110, # reference db for computing spectrogram
    'sample_rate':None, # sample rate of your data
    
    # mel filter
    'mel_filter':False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin':300, # low frequency cutoff for mel filter
    'fmax':12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,
    
    # bout searching
    'bout_auto_file':'bout_auto.pickle', # extension for saving the auto found files
    'bout_sync_file':'bout_sync.pickle', # extension for saving the synchronized auto bouts
    'stim_sync_file':'stim_sync.pickle', # extension for saving the synchronized stim if stim session
    'bout_curated_file':'bout_curated.pickle', # extension for manually curated files
    
    # if using deep_bout_search = False, the following parameters will apply for automatic bout detection:
    'read_wav_fun':bs.read_npy_chan, # function for loading the wav_like_stream (returns fs, ndarray)
    'file_order_fun':bs.sess_file_id, # function for extracting the file ID within the session
    'min_segment':20, # minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence':3000, # minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout':500, # min bout duration (ms)
    'peak_thresh_rms':0.55, # threshold (rms) for peak acceptance,
    'thresh_rms':0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh':0.3, # threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout':180000, # exclude bouts too long (ms)
    'l_p_r_thresh':100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans)
    'waveform_edges':1000, # get number of ms before and after the edges of the bout for the waveform sample
}

## other processing parameters
n_jobs = 1 # n_jobs for deriving bout info (errors when increased)
mic_file_ext = 'npy' # npy method more efficient than wav
force_preprocess = False # skip preprocessing for previously failed epochs
deep_bout_search = True # detect bouts using deep search -- see ceciestunepipe.mods.bout_detection_mf

In [3]:
# single session params
sess_par = {
    'bird':'z_r5r13_24',
    'sess':'2024-08-08',
    'stim_sess':[], # sessions where stimuli were presented
    'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
    'adc_list':[], # list of adc channels of interest
    'stim_list':['wav_stim'], # list of adc chans with the stimulus
    'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
    'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
    'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
    'on_signal':1, # sglx, whether signal on is hi or lo
    'sort':'sort_0', # sort index
    'ephys_software':'sglx'
}

## Preprocess and synchronize recordings

In [4]:
# get experiment structure
exp_struct = et.get_exp_struct(sess_par['bird'],sess_par['sess'],sort=sess_par['sort'],ephys_software=sess_par['ephys_software'])

In [5]:
##### preprocess acoustics #####
if sess_par['ephys_software'] == 'sglx':
    preproc_sglx.preprocess_session(sess_par,force_redo=True)
elif sess_par['ephys_software'] == 'oe':
    preproc_oe.preprocess_session(sess_par,force_redo=True)
print('done.')

preprocessing..
done.


In [6]:
###### derive bout information #####
sess_bout_pd = sb.get_all_day_bouts(sess_par,hparams,n_jobs=n_jobs,ephys_software=sess_par['ephys_software'],
                                    file_ext=mic_file_ext, deep_search=deep_bout_search)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

#### Process a single epoch at a time, taking care of individual errors

In [7]:
# store bout info for each epoch
bout_syn_pd_all = []
# stim sess collect all stim as well
if len(sess_par['stim_sess']) > 0:
    trial_syn_pd_all = []

In [8]:
# get epochs
sess_epochs = et.list_ephys_epochs(sess_par)
print(sess_epochs)

['0700_g0', '1000_g0', '2355_g0', '2705_g0']


### Debugging: check that all streams are the same length (address ttl events error)

In [9]:
sess_par['epoch'] = '1000_g0' # problematic epoch
epoch_struct = et.sgl_struct(sess_par,sess_par['epoch'],ephys_software=sess_par['ephys_software'])

# get epoch files
sgl_folders, sgl_files = sglu.sgl_file_struct(epoch_struct['folders']['sglx'])
run_meta_files = {k:v[0] for k,v in sgl_files.items()}
run_recordings = {k:sglex.SpikeGLXRecordingExtractor(sglu.get_data_meta_path(v)[0]) for k,v in run_meta_files.items()}

# get streams, from raw recording extractors and preprocessed data
all_streams = list(run_recordings.keys()) + ['wav'] ### might want to just remove this
# get sync pattern
all_syn_dict = {k:sy.get_syn_pattern(run_recordings,epoch_struct,k,force=False) for k in all_streams}

for stream in all_syn_dict.keys():
    time_end = np.shape(all_syn_dict[stream]['t_0'])[0]/all_syn_dict[stream]['s_f']/60
    print(stream+' recording ends at '+str(int(np.floor(time_end)))+':'+f"{round((time_end % 1)*60):02d}")

nidq recording ends at 175:20
ap_0 recording ends at 175:17
wav recording ends at 175:19


In [10]:
## debugging: check if streams have skipped heartbeats
def check_skipped(one_syn_dict: dict, round_ms=10):
    no_skips = True
    evt_arr = one_syn_dict['evt_arr']
    evt_t = one_syn_dict['t_0'][evt_arr[0]]
    print('Event array has {} events'.format(evt_arr.size//2))
    
    # get the unique periods, rounded at round_ms (default 50 ms)
    evt_period_ms = np.unique(np.round((np.unique(np.diff(evt_t))*1000)/round_ms)*round_ms).astype(int)
    if evt_period_ms.size > 1:
        evt_diff = (np.round(np.diff(evt_t)*1000/round_ms)*round_ms).astype(int)
        no_skips = False
        bad_periods = np.where(evt_diff != np.argmax(np.bincount(evt_diff)))[0]
        print('More than 1 different periods detected: {}'.format(evt_period_ms))
        print('Most periods equal to {} -- bad periods:'.format(np.argmax(np.bincount(evt_diff))))
        for bp in bad_periods:
            print('evt_diff['+str(bp)+']='+str(evt_diff[bp]))
    
    # check that the diff between every other edge is zero
    period_diff = np.hstack([np.diff(evt_arr[1][1:][::2]), np.diff(evt_arr[1][::2])])
    if not (all(period_diff==0)):
        no_skips = False
        print('Difference between corresponding periodic edges is not zero: {}'.format(np.unique(period_diff)))
    
    if no_skips: print('No skipped heartbeats')

for stream in all_syn_dict.keys():
    print('Checking for skipped heartbeats in',stream,'stream:')
    check_skipped(all_syn_dict[stream]);print()

Checking for skipped heartbeats in nidq stream:
Event array has 21040 events
More than 1 different periods detected: [130 440 500]
Most periods equal to 500 -- bad periods:
evt_diff[13797]=440
evt_diff[13798]=130

Checking for skipped heartbeats in ap_0 stream:
Event array has 21034 events
More than 1 different periods detected: [140 500]
Most periods equal to 500 -- bad periods:
evt_diff[13792]=140

Checking for skipped heartbeats in wav stream:
Event array has 21040 events
More than 1 different periods detected: [130 440 500]
Most periods equal to 500 -- bad periods:
evt_diff[13797]=440
evt_diff[13798]=130



### Need to
- cut all streams at evt_t = 13971
- resume streams at evt_t = [total length of stream] - 7240
    - 13794 for ap_0
    - 13800 for nidq
    - 13800 for wav

In [22]:
from ceciestunepipe.util import syncutil as scu

def sync_all_mismatched_streams(all_syn_dict: dict, ref_stream: str, short_stream: str, force=False) -> dict:
    ref_syn_dict = all_syn_dict[ref_stream]

    first_len_evt_arr = 13971 # number of events captured before streams were cut off
    first_len = first_len_evt_arr/2 # time when streams got cut off (s)

    last_len_evt_arr = 7240 # number of events captured after streams resumed
    last_len = last_len_evt_arr/2 # time after streams were resumed (s)
    
    # new_len = np.shape(all_syn_dict[short_stream]['t_0'])[0]/all_syn_dict[short_stream]['s_f'] # length of short stream (s)
    ref_end = int(first_len * all_syn_dict[ref_stream]['s_f']) # where to truncate ref_stream (samples)
    ref_restart = int((np.shape(all_syn_dict[ref_stream]['t_0'])[0] - last_len) * all_syn_dict[ref_stream]['s_f']) # where to restart ref_stream (samples)
    ref_len_evt_arr = np.shape(all_syn_dict[short_stream]['evt_arr'])[1] # number of events captured in ref stream
    
    for one_stream, one_syn_dict in all_syn_dict.items():
        if one_stream==ref_stream:
            continue
            
        print(' sync {}...'.format(one_stream))
        
        t_0_folder = os.path.split(one_syn_dict['t_0_path'])[0]
        t_p_path = os.path.join(t_0_folder, '{}-tp.npy'.format(one_stream))
        
        if not(os.path.exists(t_p_path) and (force is False)):
            print('  t_prime file {} not found or forced computation, getting the events'.format(t_p_path))
            
            # Edit one_stream to length of short_stream:
            one_end = int(first_len * all_syn_dict[one_stream]['s_f']) # where to truncate one_stream (samples)
            one_restart = int((np.shape(all_syn_dict[one_stream]['t_0'])[0] - last_len) * all_syn_dict[one_stream]['s_f']) # where to resume one_stream (samples)
            
            # Sync to ref stream
            one_len_evt_arr = np.shape(one_syn_dict['evt_arr'])[1]
            t_prime = scu.sync_to_pattern(np.concatenate((one_syn_dict['evt_arr'][:,:first_len_evt_arr],
                                                         one_syn_dict['evt_arr'][:,one_len_evt_arr-last_len_evt_arr:]), axis=1), 
                                         np.concatenate((one_syn_dict['t_0'][:one_end],
                                                         one_syn_dict['t_0'][one_restart:])),
                                         np.concatenate((ref_syn_dict['evt_arr'][:,:first_len_evt_arr],
                                                         ref_syn_dict['evt_arr'][:,ref_len_evt_arr-last_len_evt_arr:]), axis=1),
                                         np.concatenate((ref_syn_dict['t_0'][:ref_end],
                                                         ref_syn_dict['t_0'][ref_restart:]))
                                        )
            print('    saving t_prime array to ' + t_p_path)
            np.save(t_p_path, t_prime)
        
            # clear the memory, then load as memmap
            del t_prime
        
            one_syn_dict['t_p_path'] = t_p_path
            # save the dict with the path to the sync it
            print('    saving synced dict to {}'.format(one_syn_dict['path']))
            with open(one_syn_dict['path'], 'wb') as fp:
                pickle.dump(one_syn_dict, fp)
            
        one_syn_dict['t_p'] = np.load(t_p_path, mmap_mode='r')
    print('Done with sync_all')
    return
   

def bout_dict_from_pd_mismatched_streams(bout_pd: pd.DataFrame, all_syn_dict: dict, s_f_key: str='wav') -> dict:
    s_f = all_syn_dict[s_f_key]['s_f']

    start_ms = bout_pd['start_ms'].values
    len_ms = bout_pd['len_ms'].values
    
    bout_dict = {
        's_f': s_f, # s_f used to get the spectrogram
        's_f_nidq': all_syn_dict['nidq']['s_f'],
        's_f_ap_0': all_syn_dict['ap_0']['s_f'],
        'start_ms': start_ms,
        'len_ms': len_ms,
        'start_sample_naive': ( start_ms * s_f * 0.001).astype(np.int64),
        'start_sample_nidq': np.array([np.where(all_syn_dict['nidq']['t_0'] > start)[0][0] for start in start_ms*0.001]),
        'start_sample_wav': np.array([np.where(all_syn_dict['wav']['t_0'] > start)[0][0] for start in start_ms*0.001])
    }
    
    # Edit to remove bout starts > length of ap_0 recording
    keep = bout_dict['start_sample_wav'] <= len(all_syn_dict['wav']['t_p'])
    bout_dict['start_ms'] = bout_dict['start_ms'][keep]
    bout_dict['len_ms'] = bout_dict['len_ms'][keep]
    bout_dict['start_sample_naive'] = bout_dict['start_sample_naive'][keep]
    bout_dict['start_sample_nidq'] = bout_dict['start_sample_nidq'][keep]
    bout_dict['start_sample_wav'] = bout_dict['start_sample_wav'][keep]
    
    start_ms_ap_0 = all_syn_dict['wav']['t_p'][bout_dict['start_sample_wav']]*1000
    
    bout_dict['start_ms_ap_0'] = start_ms_ap_0
    bout_dict['start_sample_ap_0'] = np.array([np.where(all_syn_dict['ap_0']['t_0'] > start)[0][0] for start in start_ms_ap_0*0.001])
    bout_dict['start_sample_ap_0'] = (bout_dict['start_sample_ap_0']).astype(np.int64)
    bout_dict['end_sample_ap_0'] = bout_dict['start_sample_ap_0'] + (bout_dict['len_ms'] * bout_dict['s_f_ap_0'] * 0.001).astype(np.int64)
    
    ## update the bout pandas dataframe with the synced columns
    bout_pd = bout_pd.head(keep.sum()) # trim bout_pd to bouts within ap_0 recording
    for k in ['start_ms_ap_0', 'start_sample_ap_0', 'len_ms', 'start_ms', 'start_sample_naive']:
        warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
        bout_pd[k] = bout_dict[k]
        warnings.resetwarnings()

    return bout_dict, bout_pd

#### Record epochs with mismatched streams and which stream is shortest

In [12]:
mismatched_streams = {
    '0700_g0': (False,),
    '1000_g0': (True, 'ap_0'),
    '2355_g0': (False,),
    '2705_g0': (False,)
}

In [23]:
# loop through epochs:
# epoch_list = ['0700_g0', '1000_g0', '2355_g0','2705_g0']
epoch_list = ['1000_g0']

for this_epoch in epoch_list:
    
    sess_par['epoch'] = this_epoch
    epoch_struct = et.sgl_struct(sess_par,sess_par['epoch'],ephys_software=sess_par['ephys_software'])

    ##### synchronization - sglx #####
    print(sess_par['bird'],sess_par['sess'],this_epoch,'syncing..')
    if sess_par['ephys_software'] == 'sglx':
        # get epoch files
        sgl_folders, sgl_files = sglu.sgl_file_struct(epoch_struct['folders']['sglx'])
        run_meta_files = {k:v[0] for k,v in sgl_files.items()}
        run_recordings = {k:sglex.SpikeGLXRecordingExtractor(sglu.get_data_meta_path(v)[0]) for k,v in run_meta_files.items()}

        # get streams, from raw recording extractors and preprocessed data
        all_streams = list(run_recordings.keys()) + ['wav'] ### might want to just remove this
        # get sync pattern
        all_syn_dict = {k:sy.get_syn_pattern(run_recordings,epoch_struct,k,force=False) for k in all_streams}
        # run sync
        if mismatched_streams[this_epoch][0]:
            sync_all_mismatched_streams(all_syn_dict,sess_par['ref_stream'],mismatched_streams[this_epoch][1],force=False)
        else:
            sy.sync_all(all_syn_dict,sess_par['ref_stream'],force=False)

        # load bouts
        hparams, bout_pd = sb.load_bouts(sess_par['bird'],sess_par['sess'],'', derived_folder='bouts_sglx',bout_file_key='bout_auto_file')
        # keep only epoch bouts
        logger.info('bouts from this epoch {}'.format(sess_par['epoch']))
        drop_condition = ~bout_pd['file'].str.contains(sess_par['epoch'])
        bout_pd.drop(bout_pd[drop_condition].index, inplace=True)
        bout_pd.reset_index(drop=True, inplace=True)
        # sync bouts to spike time base
        bout_dict, bout_syn_pd = sy.bout_dict_from_pd(bout_pd,all_syn_dict,s_f_key='wav')
        # if mismatched_streams[this_epoch][0]:
        #     bout_dict, bout_syn_pd = bout_dict_from_pd_mismatched_streams(bout_pd,all_syn_dict,s_f_key='wav')
        # else:
        #     bout_dict, bout_syn_pd = sy.bout_dict_from_pd(bout_pd,all_syn_dict,s_f_key='wav')
        # store epoch synced bout info
        bout_syn_pd['bird'] = sess_par['bird']
        bout_syn_pd['sess'] = sess_par['sess']
        bout_syn_pd['epoch'] = sess_par['epoch']
        bout_syn_pd_all.append(bout_syn_pd)
        # save synced bouts
        bout_dict_path = os.path.join(epoch_struct['folders']['derived'],'bout_dict_ap0.pkl')
        with open(bout_dict_path, 'wb') as handle:
            pickle.dump(bout_dict, handle)
        bout_pd_path = os.path.join(epoch_struct['folders']['derived'],'bout_pd_ap0.pkl')
        bout_pd.to_pickle(bout_pd_path)
        logger.info('saved syncronized bout dict and pandas dataframe to {}, {}'.format(bout_dict_path, bout_pd_path))

        if len(sess_par['stim_sess']) > 0:
            # syn_ttl comes from the digital pin, syn_sine_ttl from the sine
            event_name = 'wav_stim'
            ttl_ev_name = event_name + '_sync_sine_ttl' 
            # get the events npy file
            npy_stim_path = os.path.join(epoch_struct['folders']['derived'],ttl_ev_name + '_evt.npy')
            stream_stim_path = os.path.join(epoch_struct['folders']['derived'],event_name + '.npy')
            trial_ttl = np.load(npy_stim_path)
            # epoch may not have trials - if so ttl file will be empty
            if len(trial_ttl) > 0:
                trial_stream = np.load(stream_stim_path,mmap_mode='r')
                # get sampling frequency
                stim_s_f = int(all_syn_dict['nidq']['s_f'])
                # load the stimulus name - frequency tag dictionary
                stim_tags_dict = preproc_sglx.load_stim_tags_dict(sess_par['stim_sess'],sess_par['bird'])
                # get trial tagged dataframe
                trial_tagged_pd = su.get_trials_pd(trial_ttl, trial_stream, stim_s_f,on_signal=sess_par['on_signal'],
                                                   tag_chan=sess_par['trial_tag_chan'],stim_tags_dict=stim_tags_dict,
                                                   trial_is_onof=True)
                # sync stim
                trial_dict, trial_syn_pd = sy.trial_syn_from_pd(trial_tagged_pd,all_syn_dict,s_f_key='nidq')
                # store epoch synced stim info
                trial_syn_pd['bird'] = sess_par['bird']
                trial_syn_pd['sess'] = sess_par['sess']
                trial_syn_pd['epoch'] = this_epoch
                trial_syn_pd_all.append(trial_syn_pd)
                # save synced stim
                stim_dict_path = os.path.join(epoch_struct['folders']['derived'],'stim_dict_ap0.pkl')
                stim_pd_path = os.path.join(epoch_struct['folders']['derived'],'stim_pd_ap0.pkl')
                with open(stim_dict_path,'wb') as handle:
                    pickle.dump(trial_dict,handle)
                trial_syn_pd.to_pickle(stim_pd_path)
                logger.info('saved syncronized stim dict and pandas dataframe to {}, {}'.format(stim_dict_path, stim_pd_path))

    ###### synchronization - oe #####
    elif sess_par['ephys_software'] == 'oe':
        # get epoch files
        run_recordings = {'oeb':preproc_oe.get_oe_cont_recording(exp_struct,this_epoch)}

        # make an all_syn_dict
        mic_file_name = os.path.join(exp_struct['folders']['derived'],this_epoch,'wav_mic-npy_meta.pickle')
        with open(mic_file_name, 'rb') as handle:
            wav_mic_meta = pickle.load(handle)
        all_syn_dict = {'wav': {'s_f':wav_mic_meta['s_f']}, 
                       'ap_0': {'s_f':run_recordings['oeb'].get_sampling_frequency()},
                       'nidq': {'s_f':run_recordings['oeb'].get_sampling_frequency()}}
        # make bouts pandas file for this session - match sglx format, streams already synced
        bout_oe_struct = et.get_exp_struct(sess_par['bird'],sess_par['sess'],sort=sess_par['sort'],ephys_software='bouts_oe')
        bout_pd_path = os.path.join(bout_oe_struct['folders']['derived'], 'bout_auto.pickle')
        bout_syn_pd = pd.read_pickle(bout_pd_path)
        bout_dict = preproc_oe.bout_dict_from_pd(bout_syn_pd,all_syn_dict)
        # store epoch synced bout info
        bout_syn_pd['bird'] = sess_par['bird']
        bout_syn_pd['sess'] = sess_par['sess']
        bout_syn_pd['epoch'] = this_epoch
        bout_syn_pd_all.append(bout_syn_pd)
        # save synced bouts
        bout_dict_path = os.path.join(epoch_struct['folders']['derived'],'bout_dict_oe.pkl')
        bout_pd_path = os.path.join(epoch_struct['folders']['derived'],'bout_pd_oe.pkl')
        with open(bout_dict_path,'wb') as handle:
            pickle.dump(bout_dict,handle)
        bout_syn_pd.to_pickle(bout_pd_path)

        if len(sess_par['stim_sess']) > 0:
            # this epoch name - get recording events path
            raw_folder = exp_struct['folders']['oe']
            epoch_path = os.path.join(raw_folder,this_epoch)
            node_path = preproc_oe.get_default_node(exp_struct,this_epoch)
            rec_path = preproc_oe.get_default_recording(node_path)
            events_path = os.path.join(rec_path,'events/Network_Events-102.0/TEXT_group_1/')
            # load stim lables / onsets
            stim_labels = np.load(os.path.join(events_path,'text.npy'))
            stim_onsets = np.load(os.path.join(events_path,'timestamps.npy'))

            # get stim onsets and offsets
            stim_on_all = []; stim_off_all = []; 
            stim_proc_path_all = []; stim_exp_path_all = [];
            stim_map_dir_all = []; stim_id_all = [];
            # loop through stim
            for stim_i in range(len(stim_labels)):
                this_stim_label = stim_labels[stim_i].astype('str')
                this_stim_onset = stim_onsets[stim_i]
                if this_stim_label[:4] == 'stim':
                    stim_exp_file = this_stim_label[5:]
                    # get stim preprocessing directory
                    stim_file_split = stim_exp_file.split('/')
                    stim_map_i = np.where([stim_file_split[i] in list(stim_map_dict.keys()) for i in range(len(stim_file_split))])[0][0]
                    stim_map_dir = stim_map_dict[stim_file_split[stim_map_i]]
                    # get remaining stim file path - identical for experiment and preprocessing
                    remaining_stim_file = '/'.join(stim_file_split[stim_map_i+1:])
                    # processing file location
                    stim_file = os.path.join(stim_map_dir,remaining_stim_file)
                    # load stim and get length
                    sf,this_wav = wavfile.read(stim_file,mmap=True)
                    stim_len = this_wav.shape[0]/sf
                    # get length of stim in samples - round up
                    stim_samp_len = int(np.ceil(stim_len * bout_dict['s_f']))
                    # get stim on / off
                    stim_on_all.append(this_stim_onset)
                    stim_off_all.append(this_stim_onset+stim_samp_len)  
                    stim_proc_path_all.append(stim_file)
                    stim_exp_path_all.append(stim_exp_file)
                    stim_map_dir_all.append(stim_map_dir)
                    stim_id_all.append(remaining_stim_file)

            # make into a pd - oe already synced
            stim_on_all_np = np.array(stim_on_all).astype('int')
            stim_off_all_np = np.array(stim_off_all).astype('int')
            stim_on_all_np_ms = 1000*(stim_on_all_np/bout_dict['s_f'])
            stim_off_all_np_ms = 1000*(stim_off_all_np/bout_dict['s_f'])
            trial_syn_pd = pd.DataFrame(np.vstack([stim_on_all_np,
                                                stim_off_all_np,
                                                stim_on_all_np_ms,
                                                stim_off_all_np_ms,
                                                stim_off_all_np_ms-stim_on_all_np_ms,
                                                stim_proc_path_all,
                                                stim_exp_path_all,
                                                stim_map_dir_all,
                                                stim_id_all]).T,
            columns=['start_sample','end_sample','start_ms','end_ms','len_ms',
                     'proc_file','exp_file','map_dir','stim_id'])
            trial_syn_pd['start_sample'] = trial_syn_pd['start_sample'].astype('int')
            trial_syn_pd['end_sample'] = trial_syn_pd['end_sample'].astype('int')
            trial_syn_pd['start_ms'] = trial_syn_pd['start_ms'].astype('float')
            trial_syn_pd['len_ms'] = trial_syn_pd['len_ms'].astype('float')
            # store epoch synced stim info
            trial_syn_pd['bird'] = sess_par['bird']
            trial_syn_pd['sess'] = sess_par['sess']
            trial_syn_pd['epoch'] = this_epoch
            trial_syn_pd_all.append(trial_syn_pd)
            trial_dict = {
                's_f': all_syn_dict['wav']['s_f'],
                'ap_0':all_syn_dict['ap_0']['s_f'],
                'nidq':all_syn_dict['nidq']['s_f'],
                'start_ms':trial_syn_pd['start_ms'],
                'len_ms':trial_syn_pd['len_ms'],
                'start_sample':trial_syn_pd['start_sample'],
                'end_sample':trial_syn_pd['end_sample'],
                'proc_file':trial_syn_pd['proc_file'],
                'exp_file':trial_syn_pd['exp_file'],
                'map_dir':trial_syn_pd['map_dir'],
                'stim_id':trial_syn_pd['stim_id']}
            # save synced stim
            stim_dict_path = os.path.join(epoch_struct['folders']['derived'],'stim_dict_ap0.pkl')
            stim_pd_path = os.path.join(epoch_struct['folders']['derived'],'stim_pd_ap0.pkl')
            with open(stim_dict_path,'wb') as handle:
                pickle.dump(trial_dict,handle)
            trial_syn_pd.to_pickle(stim_pd_path)
            print('saved syncronized stim dict and pandas dataframe to {}, {}'.format(stim_dict_path, stim_pd_path))
            
print('done.')

z_r5r13_24 2024-08-08 1000_g0 syncing..
 sync nidq...
  t_prime file /mnt/cube/chronic_ephys/der/z_r5r13_24/2024-08-08/sglx/1000_g0/nidq-tp.npy not found or forced computation, getting the events
(2, 13971)
(2, 7240)


IndexError: index 209570288 is out of bounds for axis 0 with size 209557048

### After handling all errors, save outputs and log preprocessing complete

In [ ]:
# concatenate list of synced bout data frames from each epoch and save
bout_syn_pd_all_cat = pd.concat(bout_syn_pd_all)
sb.save_auto_bouts(bout_syn_pd_all_cat,sess_par,hparams,software=sess_par['ephys_software'],bout_file_key='bout_sync_file')

# stim sess save the all sync epoch stim data frame as well
if len(sess_par['stim_sess']) > 0:
    trial_syn_pd_all_cat = pd.concat(trial_syn_pd_all)
    sb.save_auto_bouts(trial_syn_pd_all_cat,sess_par,hparams,software=sess_par['ephys_software'],bout_file_key='stim_sync_file')

# # log preprocessing complete without error
# log_dir = os.path.join('/mnt/cube/chronic_ephys/log', sess_par['bird'], sess_par['sess'])
# with open(os.path.join(log_dir,'preprocessing.log'), 'w') as f:
#     f.write(sess_par['bird']+' '+sess_par['sess']+' preprocessing complete without error\nEpochs '+', '.join(sess_epochs)+' processed\n')